In [2]:
# name: parse_html_titles
# author: Smirnov Kirill
# revision: 1.0
# revision date: 17.10.2016
# description: opens ods_file, finds urls through column, check if
#              web_page answers, gets its title and puts it to next column


# todo: open tty and start softoffice by itself
  #this way don't work: os.system("gnome-terminal -e 'bash -c soffice --accept=\"socket,host=localhost,port=2002;urp;\" --norestore --nologo --nodefault --headless'")
# todo: count rows by itself
# todo: open file with password
# todo: correct parse facebook non-unicode symbols


import requests
from lxml import html
from tqdm import tnrange
# https://github.com/seznam/pyoo
import pyoo
# to work witth pyoo, you should start its interface in tty:
# soffice --accept="socket,host=localhost,port=2002;urp;" --norestore --nologo --nodefault # --headless


# config
file_name = 'test_titles.ods'
length = 6  # number of rows in file


def get_urls(sheet):
    urls = sheet[0:length, 0].values
    print ('urls to be parsed:', urls)
    return urls


def check_url(url):
    if url.startswith('http'):
        url = url
    else:
        url = 'http://' + url
    return url


def get_page(url):
    try:
        html_page = requests.get(url)
    except:
        html_page = ''
    return html_page


def get_title(html_page):
    try:
        html_tree = html.fromstring(html_page.text)
        html_title = html_tree.xpath('//title/text()')[0]
    except:
        html_title = 'couldnt get page'
    return html_title


def parse_title(url_list):
    html_titles = []
    for i in tnrange(0, len(url_list), desc='parsing titles'):
        url = check_url(url_list[i])
        html_page = get_page(url)
        html_title = get_title(html_page)
        html_titles.append(html_title)
    print ('got titles:', html_titles)
    return (html_titles)


def write_titles(html_titles, sheet):
    sheet[0:length, 1].values = html_titles


desktop = pyoo.Desktop('localhost', 2002)
doc = desktop.open_spreadsheet(file_name)
sheet = doc.sheets[0]
urls = get_urls(sheet)
html_titles = parse_title(urls)
write_titles(html_titles, sheet)
doc.save()
doc.close()


urls to be parsed: ('www.yandex.ru', '', '', '', '', '')

got titles: ['Яндекс', 'couldnt get page', 'couldnt get page', 'couldnt get page', 'couldnt get page', 'couldnt get page']
